# Notebook responsável por apresentar os comandos básicos para manipulação de dados no mongo

## Crud com apenas 1 documento
- Inserir 1 documento **composto**
- Recuperar 1 documento **composto**
- Atualizar 1 documento **composto**
- Excluir 1 documento **composto**

## Bibliotecas importantes

In [1]:
from pymongo import MongoClient
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker


## Funções uteis

In [2]:
def criar_endereco():
    return {
        'logradouro':fake.street_name(),
        'numero':fake.building_number(),
        'bairro':fake.bairro(),
        'cidade':fake.city(),
        'uf':fake.administrative_unit(),
        'cep':fake.postcode(),
    }

In [3]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(18, 50)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
        'data_nascimento':fake.date()
    }        
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    pessoa['endereco'] = criar_endereco()

    pessoa['telefones'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['telefones'].append(fake.msisdn())

    pessoa['formacoes'] = []
    n = np.random.randint(1, 4)
    for i in range(n):
        pessoa['formacoes'].append(fake.job())
        
        
    return pessoa

## Configurações

In [4]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de 1 documento

### Conectando ao MongoDB

In [5]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_mongo02']

### Exemplo - Criação de um documento composto

In [6]:
pessoa = criar_pessoa()
pp.pprint(pessoa)

{'cpf': '643.092.578-10',
 'data_nascimento': '1990-12-08',
 'email': 'luiz felipe.cavalcanti@ig.com.br',
 'endereco': {'bairro': 'Campo Alegre',
              'cep': '66284-325',
              'cidade': 'Pires',
              'logradouro': 'Morro Valentina Silva',
              'numero': '337',
              'uf': 'Maranhão'},
 'formacoes': ['Assistente administrativo'],
 'idade': 35,
 'nome': 'Luiz Felipe Cavalcanti',
 'telefones': ['5521980850243']}


In [7]:
df = pd.DataFrame.from_records([pessoa])
df

,cpf,nome,idade,data_nascimento,email,endereco,telefones,formacoes
0,643.092.578-10,Luiz Felipe Cavalcanti,35,1990-12-08,luiz felipe.cavalcanti@ig.com.br,"{'logradouro': 'Morro Valentina Silva', 'numer...",[5521980850243],[Assistente administrativo]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cpf              1 non-null      object
 1   nome             1 non-null      object
 2   idade            1 non-null      int64 
 3   data_nascimento  1 non-null      object
 4   email            1 non-null      object
 5   endereco         1 non-null      object
 6   telefones        1 non-null      object
 7   formacoes        1 non-null      object
dtypes: int64(1), object(7)
memory usage: 192.0+ bytes


In [9]:
df.loc[0, 'data_nascimento']

'1990-12-08'

In [10]:
df.loc[0, 'endereco']

{'logradouro': 'Morro Valentina Silva',
 'numero': '337',
 'bairro': 'Campo Alegre',
 'cidade': 'Pires',
 'uf': 'Maranhão',
 'cep': '66284-325'}

In [11]:
type(df.loc[0, 'endereco'])

dict

In [12]:
df.loc[0, 'telefones']

['5521980850243']

In [13]:
type(df.loc[0, 'telefones'])

list

In [14]:
df.loc[0, 'formacoes']

['Assistente administrativo']

In [15]:
type(df.loc[0, 'formacoes'])

list

### Inserindo o documento no banco de dados

In [16]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_id = colecao_pessoa.insert_one(pessoa).inserted_id
pessoa_id

ObjectId('625ff2ad7457a0d80f095993')

## Recuperando o documento inserido

#### Recuperar pelo ID

In [17]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

{'_id': ObjectId('625ff2ad7457a0d80f095993'),
 'cpf': '643.092.578-10',
 'data_nascimento': '1990-12-08',
 'email': 'luiz felipe.cavalcanti@ig.com.br',
 'endereco': {'bairro': 'Campo Alegre',
              'cep': '66284-325',
              'cidade': 'Pires',
              'logradouro': 'Morro Valentina Silva',
              'numero': '337',
              'uf': 'Maranhão'},
 'formacoes': ['Assistente administrativo'],
 'idade': 35,
 'nome': 'Luiz Felipe Cavalcanti',
 'telefones': ['5521980850243']}


#### Recuperando por algum outro atributo

In [18]:
criterio = {
    'idade': {'$eq':pessoa['idade']}
}
pp.pprint(criterio)


{'idade': {'$eq': 35}}


In [19]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625ff24dacbe4697b5d4ac92'),
 'cpf': '705.419.326-16',
 'data_nascimento': '2003-12-28',
 'email': 'mirella.castro@uol.com.br',
 'endereco': {'bairro': 'Minas Brasil',
              'cep': '64842-749',
              'cidade': 'Azevedo da Mata',
              'logradouro': 'Travessa de Oliveira',
              'numero': '92',
              'uf': 'Sergipe'},
 'formacoes': ['Perito criminal', 'Técnico em segurança do trabalho'],
 'idade': 35,
 'nome': 'Mirella Castro',
 'telefones': ['5521929669846']}


#### Recuperando por algum outro atributo

In [20]:
criterio = {
    'nome': {'$eq':pessoa['nome']}
}
pp.pprint(criterio)


{'nome': {'$eq': 'Luiz Felipe Cavalcanti'}}


In [21]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625ff2ad7457a0d80f095993'),
 'cpf': '643.092.578-10',
 'data_nascimento': '1990-12-08',
 'email': 'luiz felipe.cavalcanti@ig.com.br',
 'endereco': {'bairro': 'Campo Alegre',
              'cep': '66284-325',
              'cidade': 'Pires',
              'logradouro': 'Morro Valentina Silva',
              'numero': '337',
              'uf': 'Maranhão'},
 'formacoes': ['Assistente administrativo'],
 'idade': 35,
 'nome': 'Luiz Felipe Cavalcanti',
 'telefones': ['5521980850243']}


In [22]:
criterio = {
    'nome': {'$eq':'nome_estranho_que_não_será_criado'}
}
pp.pprint(criterio)

{'nome': {'$eq': 'nome_estranho_que_não_será_criado'}}


In [23]:
p = colecao_pessoa.find_one(criterio)
p, type(p)

(None, NoneType)

## Atualizar o documento

In [24]:
criterio = {
    "_id":pessoa_id}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)


{'$set': {'idade': 50}}


In [25]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [26]:
# ref.: https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html
r.matched_count

1

In [27]:
r.modified_count

1

In [28]:
r.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [29]:
r.upserted_id

In [30]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

{'_id': ObjectId('625ff2ad7457a0d80f095993'),
 'cpf': '643.092.578-10',
 'data_nascimento': '1990-12-08',
 'email': 'luiz felipe.cavalcanti@ig.com.br',
 'endereco': {'bairro': 'Campo Alegre',
              'cep': '66284-325',
              'cidade': 'Pires',
              'logradouro': 'Morro Valentina Silva',
              'numero': '337',
              'uf': 'Maranhão'},
 'formacoes': ['Assistente administrativo'],
 'idade': 50,
 'nome': 'Luiz Felipe Cavalcanti',
 'telefones': ['5521980850243']}


## Atualizar o documento (que não existe)

In [31]:
criterio = {
    "_id":'123'}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)

{'$set': {'idade': 50}}


In [32]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [33]:
r.matched_count

1

In [34]:
r.modified_count

0

In [35]:
r.raw_result

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

In [36]:
r = colecao_pessoa.update_one(criterio, novo_valor, upsert=True)
r

In [37]:
r.raw_result

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

In [38]:
r.upserted_id

In [39]:
p = colecao_pessoa.find_one({"_id":r.upserted_id})
pp.pprint(p)

None


## Remover o documento

In [40]:
criterio = {
    "_id":pessoa_id}
pp.pprint(criterio)

{'_id': ObjectId('625ff2ad7457a0d80f095993')}


In [41]:
r = colecao_pessoa.delete_one(criterio)

In [42]:
r.raw_result

{'n': 1, 'ok': 1.0}

In [43]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

None
